In [39]:
import pandas as pd
import numpy as np
from rapidfuzz import fuzz
from sklearn.base import BaseEstimator, ClusterMixin
from collections import defaultdict
from tqdm import tqdm

class MemoryEfficientDBSCAN(BaseEstimator, ClusterMixin):
    def __init__(self, eps=0.5, min_samples=5):
        self.eps = eps
        self.min_samples = min_samples

    def fit(self, X):
        self.labels_ = np.full(len(X), -1)
        cluster_label = 0
        for i in tqdm(range(len(X)), desc="Clustering"):
            if self.labels_[i] != -1:
                continue
            neighbors = self._region_query(X, i)
            if len(neighbors) < self.min_samples:
                self.labels_[i] = -1  # Noise
            else:
                self._expand_cluster(X, i, neighbors, cluster_label)
                cluster_label += 1
        return self

    def _region_query(self, X, point_idx):
        return [i for i in range(len(X)) if i != point_idx and fuzz.ratio(X[point_idx], X[i]) / 100.0 >= 1 - self.eps]

    def _expand_cluster(self, X, point_idx, neighbors, cluster_label):
        self.labels_[point_idx] = cluster_label
        i = 0
        while i < len(neighbors):
            neighbor = neighbors[i]
            if self.labels_[neighbor] == -1:
                self.labels_[neighbor] = cluster_label
                new_neighbors = self._region_query(X, neighbor)
                if len(new_neighbors) >= self.min_samples:
                    neighbors.extend(new_neighbors)
            i += 1

# 1. 데이터 로드
data = pd.read_csv('community-posts.csv')

# 2. 중복 및 빈 문자열 제거
data['product_name'] = data['title'].str.strip().str.lower()
data = data[data['product_name'] != '']
data = data.drop_duplicates(subset=['product_name'])

# 3. 메모리 효율적인 DBSCAN 클러스터링
dbscan = MemoryEfficientDBSCAN(eps=0.2, min_samples=2)
labels = dbscan.fit(data['product_name'].tolist()).labels_

# 4. 클러스터링 결과를 데이터프레임에 추가
data['cluster'] = labels

# 5. 클러스터별 대표 항목 추출 함수
def get_cluster_representatives(data):
    cluster_representatives = defaultdict(list)
    
    for _, row in tqdm(data.iterrows(), total=len(data), desc="Extracting representatives"):
        cluster_id = row['cluster']
        if cluster_id != -1:
            cluster_representatives[cluster_id].append(row['product_name'])
    
    results = []
    for cluster_id, items in cluster_representatives.items():
        representative = max(items, key=lambda x: sum(fuzz.ratio(x, y) for y in items))
        results.append((cluster_id, representative, items))
    
    return results

# 6. 클러스터별 대표 항목 추출
cluster_representatives = get_cluster_representatives(data)

# 결과 출력
for cluster_id, representative, cluster_items in cluster_representatives:
    print(f"\n클러스터 {cluster_id}:")
    print(f"대표 항목: {representative}")
    print("클러스터 내 항목 수:", len(cluster_items))
    print("클러스터 내 항목 (최대 10개):")
    print(cluster_items[:10])

Extracting representatives: 100%|██████████| 96207/96207 [00:02<00:00, 45812.08it/s]



클러스터 0:
대표 항목: 하림 밥싸먹는닭가슴살 슬라이스햄
클러스터 내 항목 수: 4
클러스터 내 항목 (최대 10개):
['하림 밥싸먹는 닭가슴살 슬라이스 햄', '하림 밥싸먹는닭가슴살 슬라이스햄', '공식 하림 밥싸먹는 닭가슴살 슬라이스 햄', '하림 밥싸먹는닭가슴살슬라이스햄']

클러스터 1:
대표 항목: 베베숲 시그니처 위드블루 캡
클러스터 내 항목 수: 39
클러스터 내 항목 (최대 10개):
['베베숲 물티슈 시그니처', '베베숲 시그니처 캡', '베베숲 sm 고평량 시그니처 위드블루 캡 +휴대용', '베베숲 시그니처 위드그린 그린 휴대용', '리벤스 시그니처 물티슈', '베베숲 물티슈 시그니처캡', '베베숲 시그니처 블루 슬림 캡', '베베숲 시그니처 블루 에코 캡형 아기물티슈', '베베숲 시그니처 위드블루', '베베숲 시그니처위드블루 +그린 휴대용']

클러스터 22:
대표 항목: 청정원 파스타소스 +면
클러스터 내 항목 수: 73
클러스터 내 항목 (최대 10개):
['청정원 로제 스파게티소스 x 페이', '청정원 로제 스파게티소스', '폰타나 파스타소스 +면증정', '청정원 파스타소스', '청정원 파스타소스 x+소스 면', '청정원 비스크 로제 스파게티소스', '청정원 스파게티소스 + 면', '청정원 스파게티소스 + 스파게티면', '청정원 파스타 소스 + 스파게티면', '청정원 프리미엄파스타소스 +면 or 파우치']

클러스터 2:
대표 항목: 광동 비타500 스파클링
클러스터 내 항목 수: 37
클러스터 내 항목 (최대 10개):
['비타500제로 스파클링 유자', '광동 비타500 칼슘', '비타500 스파클링제로 미니', '비타500 제로스파클링', '비타500 제로 스파클링 유자맛', '품절 비타500 제로스파클링유자', '비타500 제로 스파클링', '광동제약 비타500', '광동 비타500', '비타500 제로 스파클링 유자']

클러스터 3:
대표 항목: 로지텍 g pro x superlight 무선게이밍마우스
클러스터 내 항목

In [37]:
conda install rapidfuzz

Channels:
 - conda-forge
 - nvidia
 - rapidsai
 - defaults
Platform: linux-64
Solving environment: done


==> WARNING: A newer version of conda exists. <==
    current version: 24.7.1
    latest version: 24.9.1

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /root/miniconda3/envs/jupyter_env

  added / updated specs:
    - rapidfuzz


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    rapidfuzz-3.10.0           |  py310hf71b8c6_0         2.0 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.0 MB

The following NEW packages will be INSTALLED:

  rapidfuzz          conda-forge/linux-64::rapidfuzz-3.10.0-py310hf71b8c6_0 



                                                                                
Preparing transaction: do